In [1]:
#1-1
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

In [2]:
#1-2
park = Customer('Park', 100)
park

Customer(name='Park', fidelity=100)

In [3]:
#1-3
class LineItem:
  """구매할 물품/갯수 생성해서 총 가격을 반환"""
  def __init__(self, product, quantity, price):
    self.product = product
    self.quantity = quantity
    self.price = price

  def total(self):
    return self.price * self.quantity

In [4]:
#1-4
class Order:
  """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
  def __init__(self, customer, cart, promotion=None):
    self.customer = customer
    self.cart = list(cart)
    self.promotion = promotion

  def total(self):
    """__total 속성이 없으면 전체 계산할 값을 계산"""
    if not hasattr(self, '__total'):
      self.__total = sum(item.total() for item in self.cart)
    return self.__total

  def due(self):
    """할인금액 차감"""
    if self.promotion is None:
      discount = 0
    else:
      discount = self.promotion.discount(self)
    return self.total() - discount

  def __repr__(self):
    fmt = '<Order total: {:.2f} due: {:.2f}>'
    return fmt.format(self.total(), self.due())

In [5]:
#1-5
class Promotion(ABC):
  """할인 혜택 클래스들의 형택를 선언"""

  @abstractmethod
  def discount(self, order):
    """할인액을 구체적인 숫자로 반환"""
    pass

In [6]:
#1-6
class FidelityPromo(Promotion):
  """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""

  def discount(self, order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
  """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""

  def discount(self, order):
    discount = 0
    for item in order.cart:
      if item.quantity >= 20:
        discount += item.total() * 0.1
    return discount

class LargeOrderPromo(Promotion):
  """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""

  def discount(self, order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
      return order.total() * 0.07
    return 0

In [7]:
#2-1
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

In [8]:
#2-2
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [9]:
#2-3
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [10]:
#2-4
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

In [11]:
#2-5
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [12]:
#2-6
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

In [13]:
#2-7
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [14]:
#2-8
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [15]:
#3-1
class Order:
  """고객(namedtuole) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
  def __init__(self, customer, cart, promotion=None):
    self.customer = customer
    self.cart = list(cart)
    self.promotion = promotion

  def total(self):
    if not hasattr(self, '__total'):
      self.__total = sum(item.total() for item in self.cart)
    return self.__total

  def due(self):
    if self.promotion is None:
      discount = 0
    else:
      discount = self.promotion(self)
    return self.total() - discount

  def __repr__(self):
    fmt = '<Order total: {:.2f} due: {:.2f}>'
    return fmt.format(self.total(), self.due())

In [16]:
#3-2
def fidelity_promo(order):
  """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
  return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
  """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * 0.1
  return discount

def large_order_promo(order):
  """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * 0.07
  return 0

In [17]:
#3-3
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [18]:
#3-4
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [19]:
#3-5
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [20]:
#3-6
Order(ann, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [21]:
#4
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
  """최대로 할인받을 금액을 반환"""
  return max(promo(order) for promo in promos)

In [22]:
#5-1
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [23]:
#5-2
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [24]:
#5-3
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [25]:
#5-4
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'ABC', 'abstractmethod', 'namedtuple', 'Customer', '_i2', 'park', '_2', '_i3', 'LineItem', '_i4', 'Order', '_i5', 'Promotion', '_i6', 'FidelityPromo', 'BulkItemPromo', 'LargeOrderPromo', '_i7', 'joe', 'ann', 'cart', '_i8', '_8', '_i9', '_9', '_i10', 'banana_cart', '_i11', '_11', '_i12', 'long_order', '_i13', '_13', '_i14', '_14', '_i15', '_i16', 'fidelity_promo', 'bulk_item_promo', 'large_order_promo', '_i17', '_17', '_i18', '_18', '_i19', '_19', '_i20', '_20', '_i21', 'promos', 'best_promo', '_i22', '_22', '_i23', '_23', '_i24', '_24', '_i25'])

In [26]:
#5-5
globals()['best_promo']

<function __main__.best_promo(order)>

In [27]:
#6
promos = [globals()[name] for name in globals() if name.endswith('_promo') and name != 'best_promo']
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]